<a href="https://www.kaggle.com/code/ahmedwael206/bigram-statistical-model-to-neural-network-model?scriptVersionId=254464800" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Step 1: Imports and Logging
import torch
import torch.nn.functional as F
import string
import os

print("[INFO] Starting Bigram Model Preparation")

# Dataset file path (update this path after uploading to Kaggle input)
dataset_path = "/kaggle/input/shakespeareonline/t8.shakespeare.txt"
assert os.path.exists(dataset_path), "[ERROR] Dataset not found!"

print(f"[INFO] Dataset path found: {dataset_path}")


In [ ]:
# Step 2: Read and preprocess dataset
with open(dataset_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

print(f"[INFO] Loaded {len(raw_text)} raw characters")

# Lowercase and tokenize by words
import re
words = re.findall(r'\b[a-zA-Z]+\b', raw_text.lower())  # extract words only
print(f"[INFO] Found {len(words)} words")

# Add start/end token '.' around each word and join them with newlines
tokenized_words = ['.' + word + '.' for word in words]
text = '\n'.join(tokenized_words)

print(f"[INFO] Tokenized and marked words (first 5 lines):")
print("\n".join(tokenized_words[:5]))


In [ ]:
# Step 3: Updated vocabulary including '.'
vocab = list(string.ascii_lowercase) + ['.']
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}

print(f"[INFO] Vocabulary size (including '.'): {len(vocab)}")
print(f"[DEBUG] stoi: {stoi}")

# Step 4: Initialize Count Matrix
count = torch.zeros((27, 27), dtype=torch.int32)
print("[INFO] Initialized 27x27 count matrix")

In [ ]:
# Step 5: Fill count matrix line-by-line (word-by-word)
for line in text.splitlines():
    for ch1, ch2 in zip(line, line[1:]):
        i = stoi[ch1]
        j = stoi[ch2]
        count[i, j] += 1

print(f"[INFO] Filled bigram counts for all tokenized words.")
print(f"[DEBUG] Count matrix sample:\n{count[:5, :5]}")


In [ ]:
# Step 5.5: Full bigram count matrix heatmap with annotations
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 14))
plt.imshow(count, cmap='Blues')
plt.colorbar(label='Count')

# Set axis labels and ticks
plt.xticks(ticks=range(len(vocab)), labels=vocab)
plt.yticks(ticks=range(len(vocab)), labels=vocab)
plt.xlabel('Next Character')
plt.ylabel('Current Character')
plt.title('Bigram Count Matrix (27x27) — Characters: a-z and .')

# Annotate every non-zero count
for i in range(len(vocab)):
    for j in range(len(vocab)):
        val = count[i, j].item()
        if val > 0:
            fontsize = 6 if val < 100 else 8 if val < 1000 else 10
            plt.text(j, i, str(val), ha='center', va='center', color='black', fontsize=fontsize)

plt.tight_layout()
plt.show()


In [ ]:
# Step 6: Normalize rows to convert counts to probabilities
prob = count.float()
row_sums = prob.sum(dim=1, keepdim=True)
prob /= row_sums + 1e-8  # avoid divide-by-zero

print(f"[INFO] Converted count matrix to probability matrix")
print(f"[DEBUG] Probability row for 'a': {prob[stoi['a']]}")


In [ ]:
# Step 7: Sample one full word from the bigram model (until '.' is generated)
def sample_word(start_char='.'):
    idx = stoi[start_char]
    word = ''
    
    print(f"[INFO] Starting generation from: '{start_char}'")
    
    while True:
        probs = prob[idx]
        next_idx = torch.multinomial(probs, num_samples=1).item()
        next_char = itos[next_idx]

        print(f"[DEBUG] '{itos[idx]}' -> '{next_char}'")
        
        if next_char == '.':
            break  # stop at end token
        word += next_char
        idx = next_idx  # continue from new char

    return word

# Generate a few sample words
print("[RESULT] Sampled words:")
for _ in range(10):
    print(sample_word())


# 📉 Limitations of Character-Level Bigram Models

A **character-level Bigram model** predicts the next character based only on the **current character**:

> **P(cₙ | cₙ₋₁)**

This means it only considers a **single character of history** when making predictions. While it's simple and fast, it leads to major limitations.

---

## ⚠️ Why Bigram Models Fail to Generalize

### 🔹 1. **Lack of Context**
The model does not account for multi-character patterns or word structures. For example, it might know that:

- `'q'` is followed by `'u'` (which is good),  
- but it **doesn't remember** what came before `'q'`.

So it might generate:

tqush
glory
blenquest


*These are examples of **hallucination** — outputs that seem plausible but aren’t real.*****

---

### 🔹 2. **One-Character Dependency**

Every character prediction depends **only** on the one before it. That means:

- It ignores all characters before `cₙ₋₁`
- It cannot enforce long-term consistency in a word
- It cannot "remember" spelling rules like `"tion"` or `"str"`

---

### 🔹 3. **Frequency Dominance & Repetition**

Because the model is trained on raw frequency counts:

```python
P(j | i) = count[i][j] / sum(count[i])


# 🤖 Why Use a Neural Network Instead of Count-Based Bigram Model?

The classic count-based bigram model estimates:

> **P(c₂ | c₁) = count[c₁][c₂] / sum(count[c₁])**

It simply memorizes how frequently each character follows another. However, this has several limitations:

---

## 🚫 Problems with Count-Based Bigram Models

- **No Generalization**: If a bigram has never occurred, its probability is 0 — even if it's linguistically valid.
- **Overfitting on Frequent Bigrams**: Common pairs like `'t' → 'h'` dominate, drowning out less common but valid patterns.
- **No Learning Mechanism**: There's no optimization or adaptation based on mistakes — it's all just raw frequency.

---

## ✅ Benefits of a Neural Bigram Model

| Feature                      | Count-Based | Neural Net |
|------------------------------|-------------|-------------|
| Learns from mistakes         | ❌          | ✅          |
| Can generalize patterns      | ❌          | ✅          |
| Differentiable and trainable | ❌          | ✅          |
| Learns low-rank structure    | ❌          | ✅          |
| Tunable via validation loss  | ❌          | ✅          |

---

## 🧪 Train/Test Split for Real Evaluation

To prevent overfitting and evaluate the model's generalization, we split our data:

- **80% for training**
- **20% for testing**

> We expect a good neural model to **perform well on unseen character pairs**, while the count-based model cannot.

---

## 📌 Conclusion

The neural model doesn't just memorize — it **learns** to smooth probabilities, generalize to similar patterns, and adapt based on training data.

This makes it far more powerful — even with the same number of parameters as a count matrix (just a 27×27 weight matrix).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import string
import random

# Vocabulary includes a-z and the special '.'
vocab = list(string.ascii_lowercase) + ['.']
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(vocab)

print(f"[INFO] Vocabulary size: {vocab_size}")


In [ ]:
# Tokenize corpus: each word becomes `.word.`
dataset_path = "/kaggle/input/shakespeareonline/t8.shakespeare.txt"
with open(dataset_path, 'r', encoding='utf-8') as f:
    raw_text = f.read().lower()
import re
words = re.findall(r'\b[a-zA-Z]+\b', raw_text)
tokens = ['.' + w + '.' for w in words]

# Collect input/output pairs from all bigrams
xs = []
ys = []

for token in tokens:
    for ch1, ch2 in zip(token, token[1:]):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])

x_tensor = torch.tensor(xs)  # input indices
y_tensor = torch.tensor(ys)  # target indices

print(f"[INFO] Prepared {len(xs)} (x, y) training pairs")


In [ ]:
# Step: Train/Test Split (80% train, 20% test)
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_tensor, y_tensor, test_size=0.2, random_state=42, shuffle=True
)

print(f"[INFO] Training samples: {x_train.shape[0]}")
print(f"[INFO] Test samples:     {x_test.shape[0]}")


In [ ]:
class BigramNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Just weights, no bias
        self.W = nn.Parameter(torch.randn(vocab_size, vocab_size))  # 27 x 27

    def forward(self, x_idx):
        # x_idx: shape [batch], containing character indices
        logits = self.W[x_idx]  # each row of W corresponds to logits for next char
        return logits

model = BigramNN(vocab_size)
print(f"[INFO] Model initialized with weight shape: {model.W.shape}")


In [ ]:
loss_fn = nn.CrossEntropyLoss()  # applies log-softmax + NLLLoss
optimizer = torch.optim.SGD(model.parameters(), lr = 0.5)

# Training step logging
def compute_loss():
    logits = model(x_train)
    loss = loss_fn(logits, y_train)
    return loss


In [ ]:
# Training loop with early stopping
for epoch in range(100000):
    loss = compute_loss()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"[EPOCH {epoch}] Loss: {loss.item():.4f}")
    # 🛑 Early stop condition
    if loss.item() < 1.0:
        print(f"[INFO] Early stopping at epoch {epoch} with loss {loss.item():.4f}")
        break


# ⚠️ Why This Neural Network Still Isn't Efficient — And What Comes Next

So far, we’ve transitioned from a **count-based bigram model** to a **neural network** that uses:

- A learned embedding for each character
- A hidden layer with non-linearity
- A softmax over the vocabulary

This model is more expressive than the raw count matrix, but **fundamentally limited in a critical way**:

---

## ❗ The Core Limitation: It's Still a Bigram Model

Even though it uses neural components, our current model still only looks at **one character at a time** to predict the next:

> **P(c₂ | c₁)** — based only on the previous single character.

It doesn’t matter whether we're using a count matrix or a neural net — the model **has no memory of what came before c₁**.

---

## 🤯 Why That’s a Problem

- **Hallucination is common**: The model may generate "realistic" looking character transitions but quickly veer off into nonsense because there's no control or grammar.
- **Misses long-term dependencies**: Words like `"follow"` require learning `'l'` after `'l'` only if it follows `'o'`, etc. Our model can't capture this.
- **Fails on spelling structure**: It can’t distinguish `"tio"` in `"nation"` vs. `"tion"` in `"action"` — both just look like random character sequences.
- **Low ceiling**: Even after extensive training, test loss rarely drops below `0.7`, because the model simply can’t “think ahead.”

---

## 🤖 Neural ≠ Intelligent (If Not Designed Right)

Just because the model uses learnable layers and activations doesn’t mean it’s "understanding" sequences. This version is just:

> A **smarter bigram table** — but still only a bigram table.

If the **input is only a single character**, the **best this network can do is what a count-based model does**, but with fancier weight tuning.

---

## 🧠 What We Need Instead: Sequence Models

To move beyond these limitations, we need to use models that can:

✅ Take **multiple previous characters** into account  
✅ Learn dependencies across **entire words or sequences**  
✅ Build an **internal state** of context while generating  

This leads us to:

### 🔄 Recurrent Neural Networks (RNNs)

- Keep a **hidden state** across time steps
- Can model **entire sequences** like `"fol"` → predict `"l"`

### ⚡ Transformers

- Use **self-attention** to see the whole context at once
- Scale better and train faster than RNNs
- Backbone of modern LLMs like GPT

---

## 🔁 Summary

| Model               | Looks Back How Far? | Learns Sequence Context? | Efficient? |
|--------------------|---------------------|---------------------------|------------|
| Count-Based Bigram | 1 char              | ❌                        | ✅          |
| Neural Bigram (ours)| 1 char             | ❌                        | ❌          |
| RNN / Transformer   | Many chars          | ✅                        | ✅✅✅        |

---

## ⏭️ Next Step

> We now need to move from **one-step character prediction** to **multi-step sequence modeling** — where the next character depends on the entire prefix.

Let’s build a **recurrent model** or a **Transformer-style character-level language model** next — one that can *truly learn to spell*.


In [ ]:
class BigramMLP(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x_idx):
        x = self.embedding(x_idx)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

mlp_model = BigramMLP(vocab_size)
mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.05)
scheduler = torch.optim.lr_scheduler.StepLR(mlp_optimizer, step_size=200, gamma=0.5)
mlp_loss_fn = nn.CrossEntropyLoss()


In [ ]:
def compute_loss_mlp(x, y):
    logits = mlp_model(x)
    return mlp_loss_fn(logits, y)

for epoch in range(10000):
    loss = compute_loss_mlp(x_train, y_train)
    mlp_optimizer.zero_grad()
    loss.backward()
    mlp_optimizer.step()
    scheduler.step()
    
    print(f"[MLP] Epoch {epoch} - Train Loss: {loss.item():.4f}")
        
    if loss.item() < 2.32:
        print(f"[MLP] Early stopping at epoch {epoch}")
        break


In [ ]:
# Evaluate on test set
with torch.no_grad():
    test_loss_mlp = compute_loss_mlp(x_test, y_test).item()
print(f"[MLP] Final Test Loss: {test_loss_mlp:.4f}")